In [1]:
import os
import requests
from bs4 import BeautifulSoup as bs

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta, date
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
import time 
import pandas as pd
from io import StringIO
import re

In [3]:
def move_to_next_page(driver):
    try:
        # 현재 페이지를 나타내는 strong 요소 찾기
        current_page = driver.find_element(By.CSS_SELECTOR, "#paging strong")
        # 다음 a 태그 찾기 (current_page 다음에 나오는 a)
        next_page = current_page.find_element(By.XPATH, "following-sibling::a[1]")
        
        # 다음 페이지가 있으면 클릭
        driver.execute_script("arguments[0].click();", next_page)
        time.sleep(3)  # 페이지가 넘어갈 시간 대기
        print("✅ 다음 페이지로 이동합니다")
        return True
    except Exception as e:
        print("✅ 다음 페이지 없음, 크롤링 종료")
        return False

In [4]:
keyword1 = input("지역을 입력해주세요")
keyword2 = input("주거용과 아파트 중 하나를 입력해주세요")

지역을 입력해주세요 경기
주거용과 아파트 중 하나를 입력해주세요 아파트


In [ ]:
str_date = input("시작일을 입력해 주세요")
end_date = input("마지막일자를 입력해 주세요")

In [ ]:

driver.find_element(By.CSS_SELECTOR, ".button.btn_white.bigrounded").click()
time.sleep(3)

# --- 경매 검색 ---
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".menuicon_10"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "siCd")))).select_by_visible_text(keyword1)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "ctgr")))).select_by_visible_text(keyword2)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd")))).select_by_visible_text('4억')

# --- 날짜 설정 ---

# str → datetime
start_dt = datetime.strptime(str_date, "%Y%m%d")
end_dt = datetime.strptime(end_date, "%Y%m%d")

# datetime → 원하는 형식의 str
start_str = start_dt.strftime("%Y-%m-%d")
end_str = end_dt.strftime("%Y-%m-%d")


wait.until(EC.element_to_be_clickable((By.ID, "bgnDt"))).send_keys(start_str)
wait.until(EC.element_to_be_clickable((By.ID, "endDt"))).send_keys(end_str)

# 면적 설정
bldgSqmEnd_input = wait.until(EC.element_to_be_clickable((By.ID, "bldgSqmEnd")))
bldgSqmEnd_input.clear()
bldgSqmEnd_input.send_keys(85)
time.sleep(1)

# 진행상태 설정
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#stat"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "stat")))).select_by_visible_text('-선택-')


# --- 검색 및 리스트 로딩 ---
wait.until(EC.element_to_be_clickable((By.ID, "btnSrch"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "dataSize_s")))).select_by_visible_text('100')
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#lsTbody > tr.hand')))
WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")

print("\n🔍 리스트 로딩 완료")

# --- 결과 저장용 ---
result1 = []
cols = ['사건번호', '낙찰가']

# --- 페이지별 데이터 수집 ---
while True:
    # 스크롤 끝까지 내려서 전체 tr 다 로딩되게 만들기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로 로딩된 tr 요소들 다시 수집
    trs = driver.find_elements(By.CSS_SELECTOR, "#lsTbody > tr.hand")
    print(f"🔵 현재 페이지 tr 개수: {len(trs)}개")

    # tr 하나하나 수집
    for idx, tr in enumerate(trs, start=1):
        try:
            tid = tr.find_element(By.CSS_SELECTOR, 'input[id^="Tid_"]').get_attribute("value")
            num = tr.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text
            final_price = int(tr.find_element(By.ID, f"sucbAmt_{tid}").text.replace(",", ""))

            # 결과 저장
            result1.append([ num, final_price ])
            print(f"✅ {idx}번째 tr 추출 완료")

            time.sleep(2)

        except Exception as e:
            print(f"⚠️ {idx}번째 tr 추출 실패: {e}")

    # 다음 페이지 없으면 break
    if not move_to_next_page(driver):
        print("✅ 다음 페이지 없음, 크롤링 종료")
        break

# --- 결과 저장 ---
df = pd.DataFrame(result1, columns=cols)
df.to_csv(f"/Users/jihyun/Desktop/자료/경매/입찰결정/[{start_dt.month:02d}월{start_dt.day:02d}일-{end_dt.month:02d}월{end_dt.day:02d}일]{keyword1}_{keyword2}_낙찰가.csv", encoding="utf-8-sig", index=False)
print("✅ 파일 저장 완료")


In [12]:
import pandas as pd

# CSV 파일 불러오기
df1 = pd.read_csv("/Users/jihyun/Desktop/자료/경매/입찰결정/[2025년04월]경기_아파트_경매.csv", encoding="utf-8-sig")  # 또는 "cp949"
df2 = pd.read_csv("/Users/jihyun/Desktop/자료/경매/입찰결정/[04월28일-04월30일]경기_아파트_낙찰가.csv", encoding="utf-8-sig")

# 'num' 컬럼 기준으로 병합 
merged_df = pd.merge(df1, df2, on='사건번호', how='left')

# 결과 확인
merged_df.head()

# 필요시 저장
merged_df.to_csv("/Users/jihyun/Desktop/자료/경매/입찰결정/병합결과.csv", encoding="utf-8-sig", index=False)


In [11]:
merged_df

,입찰여부,사건번호,주소,면적,사용승인연도,감정가,최저가,최대입찰가,입찰가,비용,실거래가,예상 낙찰비율,낙찰가_x,오늘조회수,누적조회수,입찰일,낙찰가_y
0,1,2023-8739(2),"경기 용인시 기흥구 언남동 384-2, 6층605호 (언남동,까사90) 외 1필지",49.89㎡\n(15.092평),2018,263000000,21659000,277263448.0,NaN,18069885.0,305333333,69.93%,NaN,8,931,25.04.29,NaN
1,1,2023-62211,"경기 고양시 일산서구 탄현동 1481, 403동 4층406호 (탄현동,탄현마을아파트)",68.29㎡\n(20.658평),1994,240000000,168000000,206580000.0,NaN,19920000.0,236500000,84.47%,NaN,39,906,25.04.29,225120000.0
2,1,2023-74658,"경기 파주시 문산읍 당동리 170-3, 101동 5층504호 (문산효형아파트)",59.828㎡\n(18.098평),1997,152000000,74480000,94886133.0,NaN,17197200.0,122083333,84.34%,NaN,29,"1,006",25.04.29,106660000.0
3,1,2023-79325,"경기 고양시 일산서구 주엽동 38, 201동 12층1202호 (주엽동,강선마을아파트)",101.88㎡\n(30.819평),1994,677000000,232211000,614228502.0,NaN,27438165.0,651666667,84.87%,NaN,11,584,25.04.29,NaN
4,1,2023-82683,"경기 수원시 권선구 호매실동 84-25, 114동 5층504호 (호매실동,호매실지에...",59.94㎡\n(18.132평),1998,268000000,268000000,-31840000.0,NaN,21840000.0,0,97.38%,NaN,23,259,25.04.29,274150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,1,2024-77576,"경기 양주시 장흥면 부곡리 491, 105동 6층602호 (우남송추마을)",84.9756㎡\n(25.705평),2001,269000000,92267000,200447662.0,NaN,19219005.0,229666667,63.75%,NaN,3,223,25.05.28,NaN
375,1,2024-78272,"경기 의정부시 호원동 468-4, 12층1205호 (호원동,1동)",15.64㎡\n(4.731평),2017,96000000,32928000,51807747.0,NaN,15733920.0,77541667,87.21%,NaN,4,336,25.05.28,NaN
376,1,2024-83595,"경기 의정부시 의정부동 187-5, 8층 801호 (의정부동,대상센트럴)",66.005㎡\n(19.967평),2018,267000000,186900000,213724833.0,NaN,20608500.0,244333333,54.66%,NaN,6,118,25.05.28,NaN
377,1,2024-83755,"경기 의정부시 호원동 360, 101동 11층 1101호 (호원동,호원가든아파트)",84.96㎡\n(25.7평),1997,411000000,287700000,329469500.0,NaN,24280500.0,363750000,87.21%,NaN,5,93,25.05.28,NaN
